In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#pip install pandas-profiling
import matplotlib.pyplot as plt
# pip install scikit-plot
#pip install missingno
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
%matplotlib inline

import math
import gc 
pd.options.display.max_columns = 99

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
cash = pd.read_csv('../tmp/cash.csv')
bureau = pd.read_csv('../tmp/bureau.csv')
card_credit = pd.read_csv('../tmp/card_credit.csv')
install = pd.read_csv('../tmp/install.csv')

# cash.sk_id_curr = cash.sk_id_curr.astype(str)
# bureau.sk_id_curr = bureau.sk_id_curr.astype(str)
# card_credit.sk_id_curr = card_credit.sk_id_curr.astype(str)
# install.sk_id_curr = install.sk_id_curr.astype(str)

data_application_test = pd.read_csv('../data/application_test.csv')
data_application_train = pd.read_csv('../data/application_train.csv')

# data_application_train = data_application_train.sk_id_curr.astype(str)
# data_application_test = data_application_test.sk_id_curr.astype(str)

data_pool = data_application_train.append(data_application_test,sort=False)
data_pool = reduce_mem_usage(data_pool)
del(data_application_test)
del(data_application_train)
gc.collect()
data_pool.columns = [str.lower(x) for x in data_pool.columns]

data_pool = data_pool.set_index('sk_id_curr').join(cash,how='left')
data_pool = data_pool.join(bureau.set_index('sk_id_curr'),how='left')
data_pool = data_pool.join(card_credit.set_index('sk_id_curr'),how='left')
data_pool = data_pool.join(install.set_index('sk_id_curr'),how='left')

del(cash)
del(bureau)
del(card_credit)
del(install)
gc.collect()

Memory usage of dataframe is 334.32 MB
Memory usage after optimization is: 72.03 MB
Decreased by 78.5%


35

In [4]:
data_cleaned = pd.get_dummies(data_pool)
del(data_pool)
gc.collect()
data_cleaned.drop('sk_id_curr',axis=1,inplace=True)
data_cleaned = data_cleaned.reset_index()

In [5]:
data_cleaned.head()

,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,cnt_fam_members,region_rating_client,region_rating_client_w_city,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,...,organization_type_Industry: type 8,organization_type_Industry: type 9,organization_type_Insurance,organization_type_Kindergarten,organization_type_Legal Services,organization_type_Medicine,organization_type_Military,organization_type_Mobile,organization_type_Other,organization_type_Police,organization_type_Postal,organization_type_Realtor,organization_type_Religion,organization_type_Restaurant,organization_type_School,organization_type_Security,organization_type_Security Ministries,organization_type_Self-employed,organization_type_Services,organization_type_Telecom,organization_type_Trade: type 1,organization_type_Trade: type 2,organization_type_Trade: type 3,organization_type_Trade: type 4,organization_type_Trade: type 5,organization_type_Trade: type 6,organization_type_Trade: type 7,organization_type_Transport: type 1,organization_type_Transport: type 2,organization_type_Transport: type 3,organization_type_Transport: type 4,organization_type_University,organization_type_XNA,fondkapremont_mode_not specified,fondkapremont_mode_org spec account,fondkapremont_mode_reg oper account,fondkapremont_mode_reg oper spec account,housetype_mode_block of flats,housetype_mode_specific housing,housetype_mode_terraced house,wallsmaterial_mode_Block,wallsmaterial_mode_Mixed,wallsmaterial_mode_Monolithic,wallsmaterial_mode_Others,wallsmaterial_mode_Panel,"wallsmaterial_mode_Stone, brick",wallsmaterial_mode_Wooden,emergencystate_mode_No,emergencystate_mode_Yes
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083008,0.262939,0.139404,0.024704,0.036896,0.972168,0.619141,0.014297,0.000000,0.068970,0.083313,0.125000,0.036896,0.020203,0.018997,0.000000,0.000000,0.025208,0.038300,0.972168,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311279,0.622070,NaN,0.095886,0.052887,0.984863,0.795898,0.060486,0.080017,0.034485,0.291748,0.333252,0.013000,0.077271,0.054901,0.003901,0.009804,0.092407,0.053802,0.984863,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,1.0,2,2,9,0,0,0,0,0,0,NaN,0.556152,0.729492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3039,-9832.0,-2437,NaN,1,1,0,1,0,0,2.0,2,2,17,0,0,0,0,0,0,NaN,0.650391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038,-4312.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [6]:
data_cleaned['credit_income_percent'] = data_cleaned['amt_credit'] / data_cleaned['amt_income_total']
data_cleaned['annuity_income_percent'] = data_cleaned['amt_annuity'] / data_cleaned['amt_income_total']
data_cleaned['credit_term'] = data_cleaned['amt_annuity'] / data_cleaned['amt_credit']
data_cleaned['days_employed_percent'] = data_cleaned['days_employed'] / data_cleaned['days_birth']
data_cleaned['annuity_length'] = data_cleaned['amt_credit'] / data_cleaned['amt_annuity']
data_cleaned['new_credit_to_goods_ratio'] = data_cleaned['amt_credit'] / data_cleaned['amt_goods_price']
data_cleaned['income_per_person'] = data_cleaned['amt_income_total'] / data_cleaned['cnt_fam_members']
data_cleaned['income_credit_perc'] = data_cleaned['amt_income_total'] / data_cleaned['amt_credit']
data_cleaned['annuity_income_perc'] = np.sqrt(data_cleaned['amt_annuity'] / (1 + data_cleaned['amt_income_total']))
data_cleaned['children_ratio'] = data_cleaned['cnt_children'] / data_cleaned['cnt_fam_members']
data_cleaned['inc_per_child'] = data_cleaned['amt_income_total'] / (1 + data_cleaned['cnt_children'])
data_cleaned['sources_prod'] = data_cleaned['ext_source_1'] * data_cleaned['ext_source_2'] * data_cleaned['ext_source_3']
data_cleaned['car_to_birth_ratio'] = data_cleaned['own_car_age'] / data_cleaned['days_birth']
data_cleaned['car_to_employ_ratio'] = data_cleaned['own_car_age'] / data_cleaned['days_employed']
data_cleaned['phone_to_birth_ratio'] = data_cleaned['days_last_phone_change'] / data_cleaned['days_birth']
data_cleaned['phone_to_employ_ratio'] = data_cleaned['days_last_phone_change'] / data_cleaned['days_employed'] 

#Features sans importance et comportant des valeurs aberrantes à cause des agrégation
data_cleaned.drop([ 'bureau_days_credit_enddate_sum',
         'bureau_days_enddate_fact_sum',
         'install_days_instalment_sum',
         'install_days_entry_payment_sum'],axis=1,inplace=True)

In [7]:
data_cleaned.to_csv('../tmp/data_cleaned.csv',index=False)